In [1]:
import pandas as pd
import numpy as np
import subprocess
from io import StringIO

In [77]:
project_list_str = """
dave-selfstudy01
tower-of-fantasy
"""

In [79]:
project_list = project_list_str.split()
project_list

['dave-selfstudy01', 'tower-of-fantasy']

In [43]:
project_id = 'dave-selfstudy01'
# project_id = 'tower-of-fantasy'

In [80]:
def get_all_cdn(project_id):
    get_url_cmd = """gcloud compute url-maps list \
        --format "csv[no-heading](name,DEFAULT_SERVICE,pathMatchers.defaultService)" \
        --project {}
    """.format(project_id)
    output = subprocess.getoutput(get_url_cmd)
    df_url = pd.read_csv(StringIO(output), names=['name','default_service','other_service'])
    df_url = df_url.fillna('nan')
    df_url['other_service'] = df_url.other_service.apply(lambda x: x.split('/')[-1])
    df_url['default_service'] = df_url.default_service.apply(lambda x: x.split('/')[-1])

    get_bs_cmd = """gcloud compute backend-services list \
        --format="csv[no-heading](NAME,enableCDN)" \
        --project {}
    """.format(project_id)
    output = subprocess.getoutput(get_bs_cmd)
    df_bs = pd.read_csv(StringIO(output), names=['name','enabled_cdn'])

    get_bb_cmd = """gcloud compute backend-buckets list \
        --format="csv[no-heading](NAME,enableCdn)" \
        --project {}
    """.format(project_id)
    output = subprocess.getoutput(get_bb_cmd)
    df_bb = pd.read_csv(StringIO(output), names=['name','enabled_cdn'])

    df_bbb = pd.concat([df_bs, df_bb]).reset_index(drop=True)

    services = []
    lbs = []
    for index, row in df_url.iterrows():
        services.append(row['default_service'])
        lbs.append(row['name'])
        if row['other_service'] != 'nan':
            services.append(row['other_service'])
            lbs.append(row['name'])
        else:
            pass
    
    df1 = pd.DataFrame({"services": services, "lbs": lbs})
    df_stat = pd.merge(df1, df_bbb, how='left', left_on='services', right_on='name')
    return df_stat[df_stat.enabled_cdn == True][['lbs', 'services', 'enabled_cdn']]
    

    

In [84]:
df = pd.concat([get_all_cdn(x) for x in project_list]).reset_index(drop=True)
df.head()

,lbs,services,enabled_cdn
0,http-lb01,hzchen-us,True
1,http-lb02,backend-bwglosbas01,True
2,fantasy-gcp-cdn,fantasy-copy-upload,True


# Get HTTPs LB for each project id

In [44]:
get_url_cmd = """gcloud compute url-maps list \
    --format "csv[no-heading](name,DEFAULT_SERVICE,pathMatchers.defaultService)" \
    --project {}
""".format(project_id)

In [45]:
output = subprocess.getoutput(get_url_cmd)
print(output)

http-lb01,backendBuckets/hzchen-us,https://www.googleapis.com/compute/v1/projects/dave-selfstudy01/global/backendServices/webdemo-bs
http-lb02,backendServices/backend-bwglosbas01,


In [53]:
df_url = pd.read_csv(StringIO(output), names=['name','default_service','other_service'])
df_url.head()

,name,default_service,other_service
0,http-lb01,backendBuckets/hzchen-us,https://www.googleapis.com/compute/v1/projects...
1,http-lb02,backendServices/backend-bwglosbas01,NaN


# Get backend services with CDN enabled

In [16]:
get_bs_cmd = """gcloud compute backend-services list \
    --format="csv[no-heading](NAME,enableCDN)" \
    --project {}
""".format(project_id)

In [17]:
output = subprocess.getoutput(get_bs_cmd)
print(output)

backend-bwglosbas01,True
webdemo-bs,False


In [18]:
df_bs = pd.read_csv(StringIO(output), names=['name','enabled_cdn'])
df_bs.head()

,name,enabled_cdn
0,backend-bwglosbas01,True
1,webdemo-bs,False


# Get backend buckets with CDN enabled

In [19]:
get_bb_cmd = """gcloud compute backend-buckets list \
    --format="csv[no-heading](NAME,enableCdn)" \
    --project {}
""".format(project_id)

In [20]:
output = subprocess.getoutput(get_bb_cmd)
print(output)

hzchen-us,True


In [21]:
df_bb = pd.read_csv(StringIO(output), names=['name','enabled_cdn'])
df_bb.head()

,name,enabled_cdn
0,hzchen-us,True


In [26]:
df_bbb = pd.concat([df_bs, df_bb]).reset_index(drop=True)
df_bbb.head()

,name,enabled_cdn
0,backend-bwglosbas01,True
1,webdemo-bs,False
2,hzchen-us,True


In [54]:
df_url.head()

,name,default_service,other_service
0,http-lb01,backendBuckets/hzchen-us,https://www.googleapis.com/compute/v1/projects...
1,http-lb02,backendServices/backend-bwglosbas01,NaN


In [55]:
df_url = df_url.fillna('nan')
df_url['other_service'] = df_url.other_service.apply(lambda x: x.split('/')[-1])
df_url['default_service'] = df_url.default_service.apply(lambda x: x.split('/')[-1])

In [56]:
df_url

,name,default_service,other_service
0,http-lb01,hzchen-us,webdemo-bs
1,http-lb02,backend-bwglosbas01,nan


In [57]:
for index, row in df_url.iterrows():
    print(row['default_service'], row['other_service'], row['name'])

hzchen-us webdemo-bs http-lb01
backend-bwglosbas01 nan http-lb02


In [59]:
services = []
lbs = []
for index, row in df_url.iterrows():
    services.append(row['default_service'])
    lbs.append(row['name'])
    if row['other_service'] != 'nan':
        services.append(row['other_service'])
        lbs.append(row['name'])
    else:
        pass

len(services), len(lbs)

(3, 3)

In [76]:
df1 = pd.DataFrame({"services": services, "lbs": lbs})
df_stat = pd.merge(df1, df_bbb, how='left', left_on='services', right_on='name')
df_stat[df_stat.enabled_cdn == True][['lbs', 'services', 'enabled_cdn']]

,lbs,services,enabled_cdn
0,http-lb01,hzchen-us,True
2,http-lb02,backend-bwglosbas01,True
